# Random sampling of 200 cropland raster cells: EU Crop Map 2018

This notebook reads in the EU Crop Map 2018 data and selects 200 random cropland cells for a specified NUTS2 region. All data are open source:  
- [EU Crop Map 2018](https://data.jrc.ec.europa.eu/collection/id-00346)
- [NUTS2 Boundaries](https://ec.europa.eu/eurostat/web/gisco/geodata/statistical-units/territorial-units-statistics)
- [Crop Yields](https://gaez.fao.org/)

### Read in libraries and data

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.plot import show
from rasterio import features
from rasterio.features import dataset_features
from rasterio.mask import mask
import geopandas as gpd
import time
import random
from shapely.geometry import Point

In [4]:
%%time
# Read in EU crop map 2018
eu_crop_map_full = rasterio.open("../../../Data/EU_Crop_Map/EUCROPMAP_2018.tif")

CPU times: total: 46.9 ms
Wall time: 105 ms


In [6]:
%%time
# Read in selected NUTS boundaries
NUTS_ES24 = gpd.read_file("../../../Data/Yield_Chain/random_NUTS/Selected_by_els/NUTS2_ES24.shp")
NUTS_FRC1 = gpd.read_file("../../../Data/Yield_Chain/random_NUTS/Selected_by_els/NUTS2_FRC1.shp")
NUTS_PL42 = gpd.read_file("../../../Data/Yield_Chain/random_NUTS/Selected_by_els/NUTS2_PL42.shp")
NUTS_BG31 = gpd.read_file("../../../Data/Yield_Chain/random_NUTS/Selected_by_els/NUTS2_BG31.shp")

CPU times: total: 188 ms
Wall time: 527 ms


### Tiday Data

In [8]:
%%time
# Clip EU crop map to France NUTS example
with rasterio.open("../../../Data/EU_Crop_Map/EUCROPMAP_2018.tif") as src:
        masked, transform = mask(src, NUTS_FRC1.geometry, crop=True)

CPU times: total: 10 s
Wall time: 12.6 s


In [9]:
%%time
# Reclassify raster to binary to reflect cropland 
masked_reclassified = np.where((masked > 100) & (masked < 300), 1, 0)
# Get rid of one set of square brackets
masked_reclassified = np.squeeze(masked_reclassified)

CPU times: total: 1.14 s
Wall time: 1.17 s


In [10]:
# convert numpy array to pandas dataframe
masked_reclassified_df = pd.DataFrame(masked_reclassified)

### Select 200 random raster cells and write out as new raster

In [14]:
%%time
# Select the rows and cols where raster equals cropland (1)
ones_list = np.argwhere(masked_reclassified == 1)
# Set random seed
random.seed(1)
# Select the indeces of 200 random ones
idx = random.sample(range(1, len(ones_list)), 200)
# Extract them to a list and replace inner square brackets with round brackets
selected_ones = ones_list[idx,:].tolist()
selected_ones = [tuple(i) for i in selected_ones]

CPU times: total: 3.3 s
Wall time: 3.43 s


In [16]:
%%time
# Replace all values with 0
masked_reclassified_df[:] = 0
# Set the selected positions to 1
for pos in selected_ones:
    masked_reclassified_df.iloc[pos] = 1

CPU times: total: 266 ms
Wall time: 276 ms


In [18]:
%%time
output_array = masked_reclassified_df.to_numpy()

CPU times: total: 0 ns
Wall time: 0 ns


In [20]:
np.count_nonzero(output_array)

200

In [40]:
%%time
out_meta=eu_crop_map_full.meta.copy() # copy the metadata of the source 
    
out_meta.update({
    "driver":"Gtiff",
    "height":masked.shape[1], # height starts with shape[1]
    "width":masked.shape[2], # width starts with shape[2]
    "transform":transform
})
              
with rasterio.open("../../../Data/EU_Crop_Map/first_run_prop_sampling/reclassified_for_Els/EU_CropMap_NUTS2_FRC1.tif",'w',**out_meta) as dst:
    dst.write(output_array, 1)

CPU times: total: 4.48 s
Wall time: 6.11 s


### Create a vector point for each selected random raster cell

In [22]:
# Open the raster file
with rasterio.open("../../../Data/EU_Crop_Map/first_run_prop_sampling/reclassified_for_Els/EU_CropMap_NUTS2_FRC1.tif") as src:
    # Read the first band of the raster
    raster_data = src.read(1)
    # Get the affine transformation (maps pixel coordinates to geographic coordinates)
    transform = src.transform
    # Get the width and height of a cell
    cell_width = transform.a    # Pixel width (horizontal resolution)
    cell_height = -transform.e  # Pixel height (vertical resolution, negative because y decreases)
    # Find the indices where raster value is 1
    rows, cols = (raster_data == 1).nonzero()
    # Convert the raster indices (rows, cols) to coordinates (x, y)
    points = [Point(transform * (col + 0.5, row + 0.5)) for row, col in zip(rows, cols)]

In [23]:
# Create a GeoDataFrame with the points
rand_points_gdf = gpd.GeoDataFrame(geometry=points)

# Optionally, set a CRS (Coordinate Reference System) if known
rand_points_gdf.set_crs(src.crs, inplace=True)
#rand_points_gdf.to_file("../../../Data/EU_Crop_Map/first_run_prop_sampling/reclassified_for_Els/random_points_test.gpkg")
rand_points_gdf = rand_points_gdf.to_crs('epsg:4326')

### Sample raster data from points

In [26]:
# Read in yield data
yield_barley_conv = rasterio.open("../../../Data/GAEZ/rainfed_high/reclassified_and_clipped/ycHr_brl.tif")

In [28]:
# Extract x and y of each point
coord_list = [(x, y) for x, y in zip(rand_points_gdf["geometry"].x, rand_points_gdf["geometry"].y)]

In [30]:
%%time
rand_points_gdf_yield_vals = rand_points_gdf
rand_points_gdf_yield_vals["conv_value"] = [x for x in yield_barley_conv.sample(coord_list)]
rand_points_gdf_yield_vals["conv_value"] = rand_points_gdf_yield_vals["conv_value"].str[0]

rand_points_gdf_yield_vals["org_value"] = rand_points_gdf_yield_vals["conv_value"]*0.5
#rand_points_gdf_yield_vals = rand_points_gdf_yield_vals.iloc[:, 1:]

rand_points_gdf_yield_vals.head(10)

CPU times: total: 46.9 ms
Wall time: 41 ms


,geometry,conv_value,org_value
0,POINT (3.44191 48.37425),5793.0,2896.5
1,POINT (3.36917 48.36675),8227.0,4113.5
2,POINT (3.45299 48.35163),5793.0,2896.5
3,POINT (3.31237 48.32505),6189.0,3094.5
4,POINT (3.47600 48.31635),7675.0,3837.5
5,POINT (3.07819 48.27765),8253.0,4126.5
6,POINT (3.06821 48.26565),8253.0,4126.5
7,POINT (3.57176 48.29047),6162.0,3081.0
8,POINT (3.00764 48.23888),7854.0,3927.0
9,POINT (3.43711 48.26259),7675.0,3837.5


### Results

In [32]:
conv_perc=50
org_perc=100-conv_perc

# Calculate the mean yield value, sampling the first n percentage of rows from conv organic column and then 100-n of organic 
mean_val = ((rand_points_gdf['conv_value'].head(int(round(len(rand_points_gdf)*(conv_perc/100)))).sum()) + (rand_points_gdf['org_value'].tail(int((len(rand_points_gdf)) - (round(len(rand_points_gdf)*(conv_perc/100))))).sum())) / len(rand_points_gdf)

print("Point Stats      ")
print("-----------")
print("Number of Points:", len(rand_points_gdf))
print("Number of Convent: ", int(round(len(rand_points_gdf)*(conv_perc/100))))
print("Number of Organic: ", int((len(rand_points_gdf)) - (round(len(rand_points_gdf)*(conv_perc/100)))))
print(" ")
print("Yield Vals")
print("-----------")
print("Mean Yield Value: ", mean_val)
print("Conv Yield Value: ", rand_points_gdf["conv_value"].mean())
print("Org  Yield Value: ", rand_points_gdf["org_value"].mean())

Point Stats      
-----------
Number of Points: 200
Number of Convent:  100
Number of Organic:  100
 
Yield Vals
-----------
Mean Yield Value:  6682.1425
Conv Yield Value:  9140.185
Org  Yield Value:  4570.0925
